[INDEX](Kubernetes.ipynb)  [NEXT](3.pods.ipynb)

<br/><table align="right" style='font-family:"Courier New", Courier, monospace; font-size:100%; bottom:0px;'>
    <tr><td bgcolor="#ffffff" color="#ffffff"></td><td bgcolor="#ffffff" color="#ffffff"> <img src="images/containous.logo.png" width="100" /> </td></tr></table>

<img align="left" src="https://avatars1.githubusercontent.com/u/13629408?s=400&v=4" width="60" />
## 3. Running Pods, Deployments

<pre style="align:center" >
For this section of the lab we will continue to work with our 3-node cluster comprised of 1 Master and 2 Workers.
</pre>
<img src="images/KubeNodes_1m_2w.svg.png" width="400" />
    
<br/><table align="right" style='font-family:"Courier New", Courier, monospace; font-size:100%; bottom:0px;'>
    <tr><td bgcolor="#ffffff" color="#ffffff"></td><td bgcolor="#ffffff" color="#ffffff"> <img src="images/containous.logo.png" width="100" /> </td></tr></table>

# 3. Running Pods, Deployments

<pre style="align:center" >

**In this section** we will see how we can launch pods directly from the command-line using 'kubectl run' and how it creates several resources.

This is useful for experimentation but it is not declarative.
We will also see how we can create resources from a yaml file using the 'kubectl create' command, and how we can create several containers in a Pod.

We will look at how to scale Pods, and what happens when a Pod fails
</pre>
    
<br/><table align="right" style='font-family:"Courier New", Courier, monospace; font-size:100%; bottom:0px;'>
    <tr><td bgcolor="#ffffff" color="#ffffff"></td><td bgcolor="#ffffff" color="#ffffff"> <img src="images/containous.logo.png" width="100" /> </td></tr></table>

    
# Kubernetes Pods

## Useful functions

Here we define some useful functions to be able to run this notebook in isolation:

In [1]:
. ./.shell.functions

Loading bash helper functions, new functions added:
-- checkPresent 
-- checkState 
-- cont 
-- untilNotPresent 
-- untilNotState 
-- untilPresent 
-- untilState 


In [2]:
kubectl get all

NAME             TYPE        CLUSTER-IP   EXTERNAL-IP   PORT(S)   AGE
svc/kubernetes   ClusterIP   10.96.0.1    <none>        443/TCP   1h


## Cleanup

Let's first verify that we have no Pods, Deployments, Services running in the default namespace by running the '*kubectl get all' command

Then delete any running items (except '*svc/kubernetes*') if necessary

In [3]:
kubectl get all

NAME             TYPE        CLUSTER-IP   EXTERNAL-IP   PORT(S)   AGE
svc/kubernetes   ClusterIP   10.96.0.1    <none>        443/TCP   1h


In [4]:
checkPresent pod    2-containers-in-a-pod &&
    kubectl delete po/2-containers-in-a-pod
checkPresent deploy k8s-demo              &&
    kubectl delete deploy k8s-demo
checkPresent svc    k8s-demo              &&
    kubectl delete service k8s-demo-service
cont

No resources found.
No resources found.


In [5]:
kubectl get all

NAME             TYPE        CLUSTER-IP   EXTERNAL-IP   PORT(S)   AGE
svc/kubernetes   ClusterIP   10.96.0.1    <none>        443/TCP   1h


## Deploying Pods via 'kubectl run'

<pre style="align:center" >
We've seen how we can interrogate our cluster server using the kubectl tool, obtaining information about Resources such as Pods, Nodes in the system.

Now let's create our own pods using 'kubectl run'.  We will see that this actually creates a Deployment and a ReplicaSet, as the example below:

<img src="images/KubernetesArchi_Deploy.svg.png" width="600" />

<br/><table align="right" style='font-family:"Courier New", Courier, monospace; font-size:100%; bottom:0px;'>
    <tr><td bgcolor="#ffffff" color="#ffffff"></td><td bgcolor="#ffffff" color="#ffffff"> <img src="http://www.icon100.com/up/3011/128/Twitter-alt.png" width="40"> </td><td bgcolor="#ffffff" color="#ffffff" > @containous </td></tr></table>

Normally we would not create pods directly using command-line arguments, as we will do here.

We would create objects in a repeatable, declarative way from yaml files, but we will do that later.

The below ```kubectl run``` command will create 2 pods (replicas), each running just 1 container from the k8s-demo image.

Note that we can specify labels on any kubernetes objects, using the -l option followd by key=value pairs, e.g.
    -l "key1=val1,key2=val2"
     
These labels are extremely useful allowing us to select a group of objects using a label selector.

Labels might be used to mark components of the cluster as dev, test or production for example.

Or they might be used to signal hardware specifics of a node, to distinguish physical nodes with SSD disks, or GPU which could then be prioritized for particular types of processing which might be disk i/o or CPU intensive.

k8sdemo is a small Docker image of 6MBy on the Docker hub [HERE](https://hub.docker.com/r/mjbright/k8s-demo/tags/)

<br/><table align="right" style='font-family:"Courier New", Courier, monospace; font-size:100%; bottom:0px;'>
    <tr><td bgcolor="#ffffff" color="#ffffff"></td><td bgcolor="#ffffff" color="#ffffff"> <img src="http://www.icon100.com/up/3011/128/Twitter-alt.png" width="40"> </td><td bgcolor="#ffffff" color="#ffffff" > @containous </td></tr></table>


We want to see what Resources are created, so let's just check again what exists already ... by running 'kubectl get all'


In [6]:
kubectl get all

NAME             TYPE        CLUSTER-IP   EXTERNAL-IP   PORT(S)   AGE
svc/kubernetes   ClusterIP   10.96.0.1    <none>        443/TCP   1h


Next we will run 'kubectl run' and immediately run 'kubectl get all' again.

In [7]:
kubectl run k8s-demo -l "app=k8s-demo" --image=mjbright/k8s-demo:1 --replicas=2 --port=8080
kubectl get all

deployment "k8s-demo" created
NAME              DESIRED   CURRENT   UP-TO-DATE   AVAILABLE   AGE
deploy/k8s-demo   2         2         2            0           1s

NAME                     DESIRED   CURRENT   READY     AGE
rs/k8s-demo-6b76bd84d7   2         2         0         1s

NAME              DESIRED   CURRENT   UP-TO-DATE   AVAILABLE   AGE
deploy/k8s-demo   2         2         2            0           1s

NAME                     DESIRED   CURRENT   READY     AGE
rs/k8s-demo-6b76bd84d7   2         2         0         1s

NAME                           READY     STATUS              RESTARTS   AGE
po/k8s-demo-6b76bd84d7-56b49   0/1       ContainerCreating   0          1s
po/k8s-demo-6b76bd84d7-9rfnw   0/1       ContainerCreating   0          1s

NAME             TYPE        CLUSTER-IP   EXTERNAL-IP   PORT(S)   AGE
svc/kubernetes   ClusterIP   10.96.0.1    <none>        443/TCP   1h


In the above output we see that the only running resource initially was an internal service.

Once we perform '''kubectl run'' this spawns a **deployment** called *k8s-demo*, which creates a corrsponding **replicaset** which in turn spawns 2 **pods**.

As we performed the '''kubectl get all''' immediately after the run, we see the status before the pod containers have been created ... in fact there is a delay whilst the images are downloaded (for a first run) and pods instantiated.

Re-running '''kubectl get all''' shows the pods as *running* now.

So our 'kubectl run' has created several resources as shown below:
![](images/KubernetesArchi_Deploy_k8sdemo1.svg.png)

In [8]:
untilState pod k8s-demo Running -q &&
    kubectl get all

NAME              DESIRED   CURRENT   UP-TO-DATE   AVAILABLE   AGE
deploy/k8s-demo   2         2         2            2           6s

NAME                     DESIRED   CURRENT   READY     AGE
rs/k8s-demo-6b76bd84d7   2         2         2         6s

NAME              DESIRED   CURRENT   UP-TO-DATE   AVAILABLE   AGE
deploy/k8s-demo   2         2         2            2           6s

NAME                     DESIRED   CURRENT   READY     AGE
rs/k8s-demo-6b76bd84d7   2         2         2         6s

NAME                           READY     STATUS    RESTARTS   AGE
po/k8s-demo-6b76bd84d7-56b49   1/1       Running   0          6s
po/k8s-demo-6b76bd84d7-9rfnw   1/1       Running   0          6s

NAME             TYPE        CLUSTER-IP   EXTERNAL-IP   PORT(S)   AGE
svc/kubernetes   ClusterIP   10.96.0.1    <none>        443/TCP   1h


... with more information, using option '*-o wide*'

In [9]:
kubectl get all -o wide

NAME              DESIRED   CURRENT   UP-TO-DATE   AVAILABLE   AGE       CONTAINERS   IMAGES                SELECTOR
deploy/k8s-demo   2         2         2            2           6s        k8s-demo     mjbright/k8s-demo:1   app=k8s-demo

NAME                     DESIRED   CURRENT   READY     AGE       CONTAINERS   IMAGES                SELECTOR
rs/k8s-demo-6b76bd84d7   2         2         2         6s        k8s-demo     mjbright/k8s-demo:1   app=k8s-demo,pod-template-hash=2632684083

NAME              DESIRED   CURRENT   UP-TO-DATE   AVAILABLE   AGE       CONTAINERS   IMAGES                SELECTOR
deploy/k8s-demo   2         2         2            2           6s        k8s-demo     mjbright/k8s-demo:1   app=k8s-demo

NAME                     DESIRED   CURRENT   READY     AGE       CONTAINERS   IMAGES                SELECTOR
rs/k8s-demo-6b76bd84d7   2         2         2         6s        k8s-demo     mjbright/k8s-demo:1   app=k8s-demo,pod-template-hash=2632684083

NAME              


Note in the above output we can see that our k8s-demo pods are running on kube-node-1 and kube-node-2.

## Creating resources using yaml specifications

### Obtaining yaml as output

Using the option '-o yaml' to any 'kubectl get' command we can obtain a yaml definition, so instead of

In [10]:
kubectl get deploy/k8s-demo -o wide

NAME       DESIRED   CURRENT   UP-TO-DATE   AVAILABLE   AGE       CONTAINERS   IMAGES                SELECTOR
k8s-demo   2         2         2            2           6s        k8s-demo     mjbright/k8s-demo:1   app=k8s-demo


we get ...

In [11]:
kubectl get deploy/k8s-demo -o yaml

apiVersion: extensions/v1beta1
kind: Deployment
metadata:
  annotations:
    deployment.kubernetes.io/revision: "1"
  creationTimestamp: 2018-03-18T19:58:49Z
  generation: 1
  labels:
    app: k8s-demo
  name: k8s-demo
  namespace: default
  resourceVersion: "4154"
  selfLink: /apis/extensions/v1beta1/namespaces/default/deployments/k8s-demo
  uid: c68cfe52-2ae6-11e8-acfc-02420ac00002
spec:
  replicas: 2
  selector:
    matchLabels:
      app: k8s-demo
  strategy:
    rollingUpdate:
      maxSurge: 1
      maxUnavailable: 1
    type: RollingUpdate
  template:
    metadata:
      creationTimestamp: null
      labels:
        app: k8s-demo
    spec:
      containers:
      - image: mjbright/k8s-demo:1
        imagePullPolicy: IfNotPresent
        name: k8s-demo
        ports:
        - containerPort: 8080
          protocol: TCP
        resources: {}
        terminationMessagePath: /dev/termination-log
        terminationMessagePolicy: File
      dnsPolicy: ClusterFirst
      restartPolic

## Creating resources from a yaml file

This is the usual way in which we would create Resources on our cluster as it is declarative and repeatable.

## Obtaining the example yaml

If you are using one of the VMS created for this lab then the example code will have been placed under:
~/src/git/ContainerOrchestration.Labs/Orchestration-Kubernetes/

If you are running in your own environment, or under play-with-kubernetes.com, you will first need to clone the repository:

```git clone https://github.com/ContainerOrchestration/Labs```

**NOTE**: You may also need to checkout the current branch, e.g.:FOSSAsia2018

```git checkout FOSSAsia2018```

## Inspecting the example

In the example yaml below, we see that we specify a Pod to be created (this is the smallest unit of execution in a Kubernetes cluster) which will contains 2 containers.

The yaml specifies which Kubernetes api version to use, the "*kind*" of object to be managed, and it's name.

In [12]:
cat examples/3.pod_example.yaml


apiVersion: v1
kind: Pod
metadata:
  name: 2-containers-in-a-pod
spec:
  containers:
  - name: k8s-demo
    image: mjbright/k8s-demo:1
    ports:
    - containerPort: 9876
  - name: shell
    image: alpine
    command:
      - "bin/sh"
      - "-c"
      - "while true; do date; sleep 1; done"



Let's first cleanup by verifying that this pod isn't already running.

We can use our yaml definition to delete any instances which may be running:

In [13]:
checkPresent pod 2-containers-in-a-pod && {
    echo; echo "-- Deleting pod:"
    kubectl delete -f examples/3.pod_example.yaml

    echo; echo "-- Waiting for pod to terminate:"
    untilNotPresent pod 2-containers-in-a-pod 
    }
cont

If we deleted any instances of "*2-containers-in-a-pod*" we need to wait for them to have terminated:


In [14]:
kubectl get all

NAME              DESIRED   CURRENT   UP-TO-DATE   AVAILABLE   AGE
deploy/k8s-demo   2         2         2            2           8s

NAME                     DESIRED   CURRENT   READY     AGE
rs/k8s-demo-6b76bd84d7   2         2         2         8s

NAME              DESIRED   CURRENT   UP-TO-DATE   AVAILABLE   AGE
deploy/k8s-demo   2         2         2            2           8s

NAME                     DESIRED   CURRENT   READY     AGE
rs/k8s-demo-6b76bd84d7   2         2         2         8s

NAME                           READY     STATUS    RESTARTS   AGE
po/k8s-demo-6b76bd84d7-56b49   1/1       Running   0          8s
po/k8s-demo-6b76bd84d7-9rfnw   1/1       Running   0          8s

NAME             TYPE        CLUSTER-IP   EXTERNAL-IP   PORT(S)   AGE
svc/kubernetes   ClusterIP   10.96.0.1    <none>        443/TCP   1h


Now we can create our pod as follows:

In [15]:
kubectl create -f examples/3.pod_example.yaml

pod "2-containers-in-a-pod" created


In [16]:
kubectl get po/2-containers-in-a-pod

NAME                    READY     STATUS              RESTARTS   AGE
2-containers-in-a-pod   0/2       ContainerCreating   0          0s


In [17]:
untilState pod 2-containers-in-a-pod Running &&
    kubectl get po/2-containers-in-a-pod

NAME                    READY     STATUS    RESTARTS   AGE
2-containers-in-a-pod   2/2       Running   0          5s


In [18]:
kubectl describe po/2-containers-in-a-pod

Name:         2-containers-in-a-pod
Namespace:    default
Node:         kube-node-2/10.192.0.4
Start Time:   Sun, 18 Mar 2018 19:58:57 +0000
Labels:       <none>
Annotations:  <none>
Status:       Running
IP:           10.192.3.2
Containers:
  k8s-demo:
    Container ID:   docker://d7cbed7e43c39529dcd3378ce8967530e643738c40bd7253438d62c741fbf546
    Image:          mjbright/k8s-demo:1
    Image ID:       docker-pullable://mjbright/k8s-demo@sha256:6d84d697796e0a529028808ba76e92482c4ca0c408e9f9eae8aba6f23c173264
    Port:           9876/TCP
    State:          Running
      Started:      Sun, 18 Mar 2018 19:58:59 +0000
    Ready:          True
    Restart Count:  0
    Environment:    <none>
    Mounts:
      /var/run/secrets/kubernetes.io/serviceaccount from default-token-6qqm4 (ro)
  shell:
    Container ID:  docker://18395509dbf6941d9d62a2368ad48509dc4f3fb5d9dcf103c79eb5fb96544422
    Image:         alpine
    Image ID:      docker-pullable://alpine@sha256:7b848083f93822dd21b0a2f14a11

In [19]:
kubectl exec -it 2-containers-in-a-pod -n default echo hello world

Defaulting container name to k8s-demo.
Use 'kubectl describe pod/2-containers-in-a-pod -n default' to see all of the containers in this pod.
hello world


In [20]:
kubectl logs 2-containers-in-a-pod shell | head -3

Sun Mar 18 19:59:00 UTC 2018
Sun Mar 18 19:59:01 UTC 2018
Sun Mar 18 19:59:02 UTC 2018


In [21]:
kubectl delete po/2-containers-in-a-pod

pod "2-containers-in-a-pod" deleted


In [22]:
kubectl get pods

NAME                        READY     STATUS        RESTARTS   AGE
2-containers-in-a-pod       2/2       Terminating   0          7s
k8s-demo-6b76bd84d7-56b49   1/1       Running       0          15s
k8s-demo-6b76bd84d7-9rfnw   1/1       Running       0          15s


## What happens if a pod dies?

In [23]:
kubectl get pods -o wide

NAME                        READY     STATUS        RESTARTS   AGE       IP           NODE
2-containers-in-a-pod       2/2       Terminating   0          7s        10.192.3.2   kube-node-2
k8s-demo-6b76bd84d7-56b49   1/1       Running       0          15s       10.192.2.1   kube-node-1
k8s-demo-6b76bd84d7-9rfnw   1/1       Running       0          15s       10.192.3.1   kube-node-2


When we deleted the 2-containers-in-a-pod Pod it just died.

If we delete one of our k8s-demo Pods we will see that a new Pod is created automatically, this is because it is managed by a ReplicaSet.

Running a Pod via "kubectl run" creates a Deployment which then creates a ReplicaSet.

Let's kill one of our Pods and see what happens


In [24]:
kubectl get pod | grep k8s-demo | head -1 | awk '{ print $1;}'

POD1=$(kubectl get pod | grep k8s-demo | head -1 | awk '{ print $1;}')

k8s-demo-6b76bd84d7-56b49


In [25]:
kubectl delete pod $POD1

pod "k8s-demo-6b76bd84d7-56b49" deleted


In [26]:
kubectl get pods -o wide

NAME                        READY     STATUS              RESTARTS   AGE       IP           NODE
2-containers-in-a-pod       2/2       Terminating         0          9s        10.192.3.2   kube-node-2
k8s-demo-6b76bd84d7-56b49   0/1       Terminating         0          17s       10.192.2.1   kube-node-1
k8s-demo-6b76bd84d7-9rfnw   1/1       Running             0          17s       10.192.3.1   kube-node-2
k8s-demo-6b76bd84d7-zp4vk   0/1       ContainerCreating   0          1s        <none>       kube-node-1


In [27]:
sleep 10; kubectl get pods -o wide

NAME                        READY     STATUS        RESTARTS   AGE       IP           NODE
2-containers-in-a-pod       2/2       Terminating   0          19s       10.192.3.2   kube-node-2
k8s-demo-6b76bd84d7-9rfnw   1/1       Running       0          27s       10.192.3.1   kube-node-2
k8s-demo-6b76bd84d7-zp4vk   1/1       Running       0          11s       10.192.2.2   kube-node-1


We see that our Pod was deleted but that a new one was created, this is what we expect when our Pod is managed by a ReplicaSet.

# Labels and Selectors

Labels can be applied to any Kubernetes generated objects and this can be useful to be able to apply operations to only selected objects based on their labels.

We can see that our k8s-demo Deployment earlier had labels automatically assigned to it.

We can create other Deployments, or Pods with a specific label.

In [28]:
kubectl describe deploy k8s-demo #| grep Labels:

Name:                   k8s-demo
Namespace:              default
CreationTimestamp:      Sun, 18 Mar 2018 19:58:49 +0000
Labels:                 app=k8s-demo
Annotations:            deployment.kubernetes.io/revision=1
Selector:               app=k8s-demo
Replicas:               2 desired | 2 updated | 2 total | 2 available | 0 unavailable
StrategyType:           RollingUpdate
MinReadySeconds:        0
RollingUpdateStrategy:  1 max unavailable, 1 max surge
Pod Template:
  Labels:  app=k8s-demo
  Containers:
   k8s-demo:
    Image:        mjbright/k8s-demo:1
    Port:         8080/TCP
    Environment:  <none>
    Mounts:       <none>
  Volumes:        <none>
Conditions:
  Type           Status  Reason
  ----           ------  ------
  Available      True    MinimumReplicasAvailable
OldReplicaSets:  <none>
NewReplicaSet:   k8s-demo-6b76bd84d7 (2/2 replicas created)
Events:
  Type    Reason             Age   From                   Message
  ----    ------             ----  ----            

Now let's create a 2nd deployment with a label we assign:

In [29]:
kubectl run k8s-demo-test -l "mylabel=testing-v3" --image=mjbright/k8s-demo:3

deployment "k8s-demo-test" created


In [30]:
kubectl get po | grep k8s-demo

k8s-demo-6b76bd84d7-9rfnw       1/1       Running             0          28s
k8s-demo-6b76bd84d7-zp4vk       1/1       Running             0          12s
k8s-demo-test-c5d55945b-wzfnl   0/1       ContainerCreating   0          1s


Now we can choose to operate on, e.g. delete, only objects matching a label - note we must delete the Deployment itself no the Pods else they will be recreated.

In [31]:
kubectl delete deploy -l "mylabel=testing-v3"

deployment "k8s-demo-test" deleted


In [32]:
kubectl get po | grep k8s-demo

k8s-demo-6b76bd84d7-9rfnw   1/1       Running       0          31s
k8s-demo-6b76bd84d7-zp4vk   1/1       Running       0          15s


This is an extremely important concept in Kubernetes allowing a loose coupling of elements, where Controllers can keep track of elements of the cluster and request resources based on their labels.

## What happens if a node dies?
We should see similar behaviour is a Node dies.

Anyone want to try this?

# Scaling our application

As our Pods are managed by a **ReplicaSet** we can scale the number of replicas by using the scale command on the **Deployment**:

In [33]:
kubectl scale --replicas=3 deploy/k8s-demo

deployment "k8s-demo" scaled


In [34]:
kubectl get all

NAME              DESIRED   CURRENT   UP-TO-DATE   AVAILABLE   AGE
deploy/k8s-demo   3         3         3            2           33s

NAME                     DESIRED   CURRENT   READY     AGE
rs/k8s-demo-6b76bd84d7   3         3         2         33s

NAME              DESIRED   CURRENT   UP-TO-DATE   AVAILABLE   AGE
deploy/k8s-demo   3         3         3            2           33s

NAME                     DESIRED   CURRENT   READY     AGE
rs/k8s-demo-6b76bd84d7   3         3         2         33s

NAME                           READY     STATUS              RESTARTS   AGE
po/2-containers-in-a-pod       2/2       Terminating         0          25s
po/k8s-demo-6b76bd84d7-9rfnw   1/1       Running             0          33s
po/k8s-demo-6b76bd84d7-qg982   0/1       ContainerCreating   0          0s
po/k8s-demo-6b76bd84d7-zp4vk   1/1       Running             0          17s

NAME             TYPE        CLUSTER-IP   EXTERNAL-IP   PORT(S)   AGE
svc/kubernetes   ClusterIP   10.96.0.1    

In [35]:
kubectl get pods -o wide

NAME                        READY     STATUS              RESTARTS   AGE       IP           NODE
2-containers-in-a-pod       2/2       Terminating         0          26s       10.192.3.2   kube-node-2
k8s-demo-6b76bd84d7-9rfnw   1/1       Running             0          34s       10.192.3.1   kube-node-2
k8s-demo-6b76bd84d7-qg982   0/1       ContainerCreating   0          1s        <none>       kube-node-1
k8s-demo-6b76bd84d7-zp4vk   1/1       Running             0          18s       10.192.2.2   kube-node-1


### Scaling down

We can of course scale back down:

In [36]:
kubectl scale --replicas=2 deploy/k8s-demo

deployment "k8s-demo" scaled


In [37]:
untilNotState pod k8s-demo Terminating

k8s-demo-6b76bd84d7-qg982   0/1       Terminating   0          2s
k8s-demo-6b76bd84d7-qg982   0/1       Terminating   0          11s


In [38]:
kubectl get pods -o wide

NAME                        READY     STATUS        RESTARTS   AGE       IP           NODE
2-containers-in-a-pod       0/2       Terminating   0          44s       10.192.3.2   kube-node-2
k8s-demo-6b76bd84d7-9rfnw   1/1       Running       0          52s       10.192.3.1   kube-node-2
k8s-demo-6b76bd84d7-zp4vk   1/1       Running       0          36s       10.192.2.2   kube-node-1


## Accessing our application

To access our application pods we should expose them as a service.
This would have the advantage of being able to address a virtual ip address for our service as pods themselves come and go (so we don't need to determine their address), and also allows load balancing between pods.

### Accessing our application - directly accessing Pods

But there are other more basic ways that we will investigate first.

If we perform '''kubectl get pods -o wide''' we will see the cluster ip of our pods, enabling us to access directly to the pods.

In [39]:
kubectl get pods -o wide 

NAME                        READY     STATUS        RESTARTS   AGE       IP           NODE
2-containers-in-a-pod       0/2       Terminating   0          45s       10.192.3.2   kube-node-2
k8s-demo-6b76bd84d7-9rfnw   1/1       Running       0          53s       10.192.3.1   kube-node-2
k8s-demo-6b76bd84d7-zp4vk   1/1       Running       0          37s       10.192.2.2   kube-node-1


In [40]:
POD1=$(kubectl get pod -o wide --no-headers | grep k8s-demo | head -1 | awk '{print $1;}')
POD2=$(kubectl get pod -o wide --no-headers | grep k8s-demo | tail -1 | awk '{print $1;}')
POD1_IP=$(kubectl get pod -o wide --no-headers | grep k8s-demo | head -1 | awk '{print $6;}')
POD2_IP=$(kubectl get pod -o wide --no-headers | grep k8s-demo | tail -1 | awk '{print $6;}')

echo POD1=$POD1 POD1_IP=$POD1_IP
echo POD2=$POD2 POD2_IP=$POD2_IP

POD1=k8s-demo-6b76bd84d7-9rfnw POD1_IP=10.192.3.1
POD2=k8s-demo-6b76bd84d7-zp4vk POD2_IP=10.192.2.2


Let's look at the yaml information for POD1:

In [41]:
kubectl get pod $POD1 -o yaml

apiVersion: v1
kind: Pod
metadata:
  creationTimestamp: 2018-03-18T19:58:49Z
  generateName: k8s-demo-6b76bd84d7-
  labels:
    app: k8s-demo
    pod-template-hash: "2632684083"
  name: k8s-demo-6b76bd84d7-9rfnw
  namespace: default
  ownerReferences:
  - apiVersion: extensions/v1beta1
    blockOwnerDeletion: true
    controller: true
    kind: ReplicaSet
    name: k8s-demo-6b76bd84d7
    uid: c68ec24a-2ae6-11e8-acfc-02420ac00002
  resourceVersion: "4146"
  selfLink: /api/v1/namespaces/default/pods/k8s-demo-6b76bd84d7-9rfnw
  uid: c69250c2-2ae6-11e8-acfc-02420ac00002
spec:
  containers:
  - image: mjbright/k8s-demo:1
    imagePullPolicy: IfNotPresent
    name: k8s-demo
    ports:
    - containerPort: 8080
      protocol: TCP
    resources: {}
    terminationMessagePath: /dev/termination-log
    terminationMessagePolicy: File
    volumeMounts:
    - mountPath: /var/run/secrets/kubernetes.io/serviceaccount
      name: default-token-6qqm4
      readOnly: true
  dnsPolicy: ClusterFirst
  n

We can see that the containerPort 8080/TCP is exposed (as we specified on our initial '*kubectl run*' command).

So now we can access to the pods directly on their port 8080.

In [42]:
curl http://$POD1_IP:8080




                               
                    (((((((((                  
               .(((((((((((((((((.             
           .((((((((((((&((((((((((((.         
       /((((((((((((((((@((((((((((((((((/     
      ((((((((((((((((((@((((((((((((((((((    
     *(((((##((((((@@@@@@@@@@@((((((%#(((((*   
     (((((((@@@(@@@@#((@@@((#@@@@(@@@(((((((   
    *(((((((((@@@@(((((@@@(((((@@@@(((((((((,  
    (((((((((@@@%@@@@((@@@((@@@@%@@@(((((((((  
   .(((((((((@@((((@@@@@@@@@@@((((@@(((((((((. 
   (((((((((&@@(((((@@@(((@@@(((((@@&((((((((( 
   (((((((((&@@@@@@@@@@#(#@@@@@@@@@@&((((((((( 
  ((((((@@@@@@@@(((((@@@@@@@(((((&@@@@@@@((((((
  (((((((((((%@@((((%@@@(@@@%((((@@&(((((((((((
   ((((((((((((@@@((@@%(((%@@((@@@(((((((((((( 
     (((((((((((#@@@@%(((((&@@@@#(((((((((((   
      /(((((((((((@@@@@@@@@@@@@(((((((((((/    
        (((((((((@@(((((((((((@@(((((((((      
          (((((((&(((((((((((((&(((((((        
           /(((((((((((((((((((((((((/         
     

In [43]:
curl http://$POD2_IP:8080




                               
                    (((((((((                  
               .(((((((((((((((((.             
           .((((((((((((&((((((((((((.         
       /((((((((((((((((@((((((((((((((((/     
      ((((((((((((((((((@((((((((((((((((((    
     *(((((##((((((@@@@@@@@@@@((((((%#(((((*   
     (((((((@@@(@@@@#((@@@((#@@@@(@@@(((((((   
    *(((((((((@@@@(((((@@@(((((@@@@(((((((((,  
    (((((((((@@@%@@@@((@@@((@@@@%@@@(((((((((  
   .(((((((((@@((((@@@@@@@@@@@((((@@(((((((((. 
   (((((((((&@@(((((@@@(((@@@(((((@@&((((((((( 
   (((((((((&@@@@@@@@@@#(#@@@@@@@@@@&((((((((( 
  ((((((@@@@@@@@(((((@@@@@@@(((((&@@@@@@@((((((
  (((((((((((%@@((((%@@@(@@@%((((@@&(((((((((((
   ((((((((((((@@@((@@%(((%@@((@@@(((((((((((( 
     (((((((((((#@@@@%(((((&@@@@#(((((((((((   
      /(((((((((((@@@@@@@@@@@@@(((((((((((/    
        (((((((((@@(((((((((((@@(((((((((      
          (((((((&(((((((((((((&(((((((        
           /(((((((((((((((((((((((((/         
     

**Note:** In a real deployment of course the actual Pod ips would not be externally accessible so we would want some way of making the nodes available on a well known address.